In [1]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

In [34]:
# Non active InsecureRequestWarning
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
# Turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Create range of Province and Pages
provinces = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 99]  # Daftar provinsi
pages = list(range(1, 3))  # Make sure big value is 3940

In [4]:
# set base url
base_url = os.getenv('BASE_URL_KEMPERIN')
# Save and load csv files to scrapped
directory = "data"

In [5]:
# extract each html page
def extract_html_page(province, page):
    data = {'id': [], 'name': [], 'address': [], 'phone': [], 'kbli': []}
    url = f"{base_url}?what=&prov={province}&hal={page}"
    
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        return data
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return data

    soup = BeautifulSoup(response.text, 'html.parser')
    div_tb = soup.find('div', {'class':'col-md-12 col-lg-12 col-xs-12 col-sm-12'})
    if not div_tb:
        print(f"No tag div in page {url}")
        return data

    table = soup.find('table', {'id': 'newspaper-a'})
    if not table:
        print(f"No table in page {url}")
        return data

    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) < 3:
            continue

        establishment, address, phone = extract_establishment_info(cells[1])
        kbli = cells[2].text.strip()
        index_number = cells[0].text.strip().replace('.', '')

        data['id'].append(int(index_number))
        data['name'].append(establishment)
        data['address'].append(address)
        data['phone'].append(phone)
        data['kbli'].append(kbli)

    return data

# extract each establishment
def extract_establishment_info(cell):
    establishment = cell.find('b').text.strip()
    full_address = cell.find_all(text=True, recursive=False)
    address = full_address[0].strip() if full_address else ''
    phone = full_address[1].strip() if len(full_address) > 1 else ''
    return establishment, address, phone

# Save data to CSV file
def save_to_csv(data, province, page):
    df = pd.DataFrame(data)
    path_file = os.path.join(f'{directory}/raw', f'province_{province}_page_{page}.csv')
    df.to_csv(path_file, index=False)
    print(f"Data page 1 to {page} for province {province} saved to {path_file}")

# Loop through each province and page and save data to CSV files
def generate(provinces, pages):
    for province in tqdm(provinces):
        for idx, page in enumerate(pages, start=1):
            data = extract_html_page(province, page)

            # Save data for every 100 pages
            if idx % 100 == 0:
                save_to_csv(data, province, page)
                
            # Save if not saved yet
            if data['id']:
                save_to_csv(data, province, page)
    
    print("Scraping finished!")

In [6]:
generate(provinces, pages)

  0%|          | 0/39 [00:00<?, ?it/s]C:\Users\mreko\AppData\Local\Temp\ipykernel_40816\2248647862.py:47: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  full_address = cell.find_all(text=True, recursive=False)


Data page 1 to 1 for province 1 saved to data/raw\province_1_page_1.csv


2it [00:00,  2.55it/s]
  3%|▎         | 1/39 [00:00<00:29,  1.27it/s]

Data page 1 to 2 for province 1 saved to data/raw\province_1_page_2.csv


Data page 1 to 1 for province 2 saved to data/raw\province_2_page_1.csv


...

2it [00:00,  2.79it/s]
 97%|█████████▋| 38/39 [00:47<00:00,  1.25it/s]

Data page 1 to 2 for province 38 saved to data/raw\province_38_page_2.csv


Data page 1 to 1 for province 99 saved to data/raw\province_99_page_1.csv


2it [00:00,  2.82it/s]
100%|██████████| 39/39 [00:47<00:00,  1.23s/it]

Scraping finished!
